In [ ]:
import plotly.graph_objects as go
import pandas as pd
import requests
import streamlit as st

# -----------------------------------------------------------------------------
# 1. CONFIGURACIÓN Y CONTEXTO
# -----------------------------------------------------------------------------
# Recuperamos variables del contexto de app.py
palette = locals().get("active_palette", ["#004d40", "#889064"]) # Default fallback colors
active_font = locals().get("active_font", "sans-serif")

# Asignación de colores
COLOR_TOTAL = palette[0]
COLOR_PC = palette[1] if len(palette) > 1 else palette[0]

# -----------------------------------------------------------------------------
# 2. FUNCIONES DE DATOS (CON CACHÉ)
# -----------------------------------------------------------------------------

@st.cache_data(show_spinner=False)
def obtener_tipo_cambio_actual():
    """Obtiene el TC actual para convertir dólares a pesos."""
    try:
        url = "https://open.er-api.com/v6/latest/USD"
        data = requests.get(url, timeout=5).json()
        mxn_rate = data['rates']['MXN']
        return mxn_rate
    except Exception as e:
        # Fallback silencioso seguro
        return 20.0

@st.cache_data(show_spinner=False)
def obtener_datos_convertidos():
    """Descarga datos del Banco Mundial y los convierte a MXN."""
    headers = {'User-Agent': 'Mozilla/5.0'}

    try:
        # 1. Tasa de cambio
        tc_actual = obtener_tipo_cambio_actual()

        # 2. Descarga Banco Mundial (PIB Total y Per Cápita)
        url_base = "https://api.worldbank.org/v2/country/MX/indicator"
        params = {'format': 'json', 'date': '2005:2024', 'per_page': 100}

        # Requests secuenciales (podrían ser paralelas, pero secuencial es más robusto aquí)
        r_total = requests.get(f"{url_base}/NY.GDP.MKTP.CD", params=params, headers=headers, timeout=10)
        r_pc = requests.get(f"{url_base}/NY.GDP.PCAP.CD", params=params, headers=headers, timeout=10)

        data_total = r_total.json()
        data_pc = r_pc.json()

        # Validación básica de respuesta API
        if len(data_total) < 2 or len(data_pc) < 2:
            return pd.DataFrame(), 0

        # 3. Procesamiento
        df1 = pd.DataFrame(data_total[1])[['date', 'value']].rename(columns={'value': 'PIB_Total_USD'})
        df2 = pd.DataFrame(data_pc[1])[['date', 'value']].rename(columns={'value': 'PIB_Per_Capita_USD'})

        df = pd.merge(df1, df2, on='date').dropna()
        df['Año_Texto'] = df['date'].astype(str)
        df = df.sort_values('date')

        # --- CONVERSIÓN ---
        # Billones = Miles de Millones (1e9)
        df['PIB_Total_MXN_Billones'] = (df['PIB_Total_USD'] * tc_actual) / 1e9
        df['PIB_Per_Capita_MXN'] = df['PIB_Per_Capita_USD'] * tc_actual

        # Índice auxiliar para eje X
        df['Posicion_X'] = range(len(df))

        return df, tc_actual

    except Exception as e:
        st.error(f"Error conectando con Banco Mundial: {e}")
        return pd.DataFrame(), 0

# -----------------------------------------------------------------------------
# 3. GENERACIÓN DE GRÁFICAS
# -----------------------------------------------------------------------------

def plot_bar_chart(df, y_col, title, color_bar, y_title, format_str, suffix=""):
    """Función genérica para crear las gráficas con estilo consistente."""

    # Datos del último punto para anotación
    ultimo_x = df['Posicion_X'].iloc[-1]
    ultimo_val = df[y_col].iloc[-1]
    anio = df['Año_Texto'].iloc[-1]

    fig = go.Figure()

    # Barras
    fig.add_trace(go.Bar(
        x=df['Posicion_X'],
        y=df[y_col],
        marker_color=color_bar,
        name=title
    ))

    # Anotación 1: Flecha con el valor actual (en la gráfica)
    label_text = f"{anio}<br>${ultimo_val:,.0f} {suffix}"
    fig.add_annotation(
        x=ultimo_x, y=ultimo_val,
        text=label_text,
        showarrow=True, arrowhead=2, ax=0, ay=-40,
        bgcolor="white", bordercolor=color_bar,
        font=dict(color="black")
    )

    # ---------------------------------------------------------
    # NUEVA ANOTACIÓN: Fuente de datos (Inferior Izquierda)
    # ---------------------------------------------------------
    fig.add_annotation(
        text="Fuente: Banco Mundial / Open Exchange Rates",
        xref="paper", yref="paper", # Coordenadas relativas al contenedor (0 a 1)
        x=0,          # 0 = Izquierda total
        y=-0.2,       # Valor negativo para estar debajo del eje X
        showarrow=False,
        xanchor='left',
        yanchor='top',
        font=dict(size=10, color="gray")
    )

    # Layout
    fig.update_layout(
        title=dict(
            text=title,
            x=0.5,
            xanchor='center',
            font=dict(size=18)
        ),
        font=dict(family=active_font),
        yaxis=dict(
            title=y_title,
            tickformat=format_str,
            gridcolor='#eee'
        ),
        xaxis=dict(
            tickmode='array',
            tickvals=df['Posicion_X'],
            ticktext=df['Año_Texto'],
            title='',
            showgrid=False
        ),
        plot_bgcolor='white',
        # Aumentamos el margen inferior (b) de 40 a 80 para que quepa la fuente
        margin=dict(t=60, b=80), 
        height=500
    )

    st.plotly_chart(fig, use_container_width=True)


# -----------------------------------------------------------------------------
# 4. EJECUCIÓN PRINCIPAL
# -----------------------------------------------------------------------------

st.markdown("### 💰 Evolución del PIB en México")

with st.spinner("Obteniendo datos actualizados del Banco Mundial..."):
    df, tc = obtener_datos_convertidos()

if not df.empty:
    st.caption(f"ℹ️ Cifras convertidas a Pesos Mexicanos (MXN) usando Tasa de Cambio referencial de ${tc:.2f}")

    tab_total, tab_pc = st.tabs(["🏛️ PIB Total Nacional", "👤 PIB Per Cápita"])

    with tab_total:
        plot_bar_chart(
            df,
            'PIB_Total_MXN_Billones',
            f'PIB Nacional Total (MXN)',
            COLOR_TOTAL,
            'Miles de Millones de Pesos',
            ',.0f',
            "MM"
        )

    with tab_pc:
        plot_bar_chart(
            df,
            'PIB_Per_Capita_MXN',
            'PIB Per Cápita (MXN)',
            COLOR_PC,
            'Pesos Mexicanos',
            ',.0f',
            ""
        )
else:
    st.warning("No se pudieron recuperar los datos del PIB en este momento.")

Recuperando datos históricos del Banco Mundial...
Tipo de cambio actual obtenido: 1 USD = $18.295137 MXN (Tue, 02 De)
Generando gráficas en MXN (TC: 18.295137)...
